V2: 无数据泄露的分层KFOLD划分         
V3: mask_contour数据增强的debug         
V4: 输出训练集和public测试集的mask_contour,但没有划分CV

In [ ]:
import numpy as np 
import pandas as pd 
import os
from sklearn.cluster import KMeans
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from collections import Counter, defaultdict
from sklearn.utils import check_random_state

## Params

In [ ]:
seed = 35#32
nfold = 5

# Augmentation of lung mask

In [ ]:
# First, installing the dependencies
!pip install -U ../input/kerasapplications/Keras_Applications-1.0.8-py3-none-any.whl
!pip install ../input/qubvel/efficientnet-1.0.0-py3-none-any.whl
!pip install ../input/qubvel/image_classifiers-1.0.0-py3-none-any.whl

# Now, installing segmentation_models (short for 'sm')
!pip install ../input/qubvel-segmentation-model-keras-v101/segmentation_models-master

# sm can work with both Keras and Tensorflow.
# By default, it look for keras.
# But, with Keras, it's giving error during the import. 
# So, we will be using Tensorflow as the backend for sm.
%env SM_FRAMEWORK=tf.keras

In [ ]:
import numpy as np
import pandas as pd 
import cv2
import os
import matplotlib.pyplot as plt

from keras import backend as K
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

import keras
import json
from tqdm import tqdm
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score
import gc
from segmentation_models import Unet
import segmentation_models  as sm
from sklearn.model_selection import train_test_split
from keras.utils import Sequence
from keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')
print(os.listdir('../input'))

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    
from pathlib import Path
import ast
from PIL import Image  

In [ ]:
ctr = pd.read_csv('../input/ranzcr-clip-lung-contours/RANZCR_CLiP_lung_contours.csv')
train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [ ]:
DIMENSION =  (512, 512) #(128, 128)  #
IMG_HEIGHT, IMG_WIDTH = DIMENSION

BACKBONE = 'seresnet34'
TRAIN_PATH = '../input/ranzcr-clip-catheter-line-classification/train/'
TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test/'
# all_images = os.listdir(TRAIN_PATH)
# all_images = [Path(e).stem for e in all_images]

In [ ]:
def load_mask(StudyInstanceUID):
    img = cv2.imread(TRAIN_PATH+StudyInstanceUID+'.jpg',1)
    ctr_left = ast.literal_eval(ctr.loc[ctr.StudyInstanceUID==StudyInstanceUID,'left_lung_contour'].values[0])
    ctr_right = ast.literal_eval(ctr.loc[ctr.StudyInstanceUID==StudyInstanceUID,'right_lung_contour'].values[0])
    img = cv2.drawContours(img, np.array([[np.array(x) for x in ctr_left]]), 0, (255,255,255), 1)
    img = cv2.drawContours(img, np.array([[np.array(x) for x in ctr_right]]), 0, (255,255,255), 1)
#     img = np.where(img>=255, 1.0, 0.0)
    return img

In [ ]:
StudyInstanceUID = '1.2.826.0.1.3680043.8.498.10000428974990117276582711948006105617'# train.loc[0,'StudyInstanceUID']
img = cv2.imread(TRAIN_PATH+StudyInstanceUID+'.jpg',1)
mask_img = load_mask(StudyInstanceUID)
fix, ax = plt.subplots(2,2, figsize=(10,10))
for i in range(2):
    ax[i,0].imshow(img)
    ax[i,1].imshow(mask_img)
    ax[i,0].axis('off')
    ax[i,1].axis('off')
plt.show()

In [ ]:
plt.imshow(cv2.imread('../input/ranzcr-clip-lung-contours/train_lung_masks/train_lung_masks/1.2.826.0.1.3680043.8.498.10000428974990117276582711948006105617.jpg'))
plt.show()

In [ ]:
from time import time

In [ ]:
start = time()
for tra_idx in tqdm(range(len(train))):
    StudyInstanceUID = train.loc[tra_idx,'StudyInstanceUID']
    #img = cv2.imread(TRAIN_PATH+StudyInstanceUID+'.jpg',1)
    mask_img = load_mask(StudyInstanceUID)
    cv2.imwrite('./train_ctrlung/'+StudyInstanceUID+'.jpg', mask_img)
print('draw contours time =',time()-start)

In [ ]:
model = Unet(backbone_name=BACKBONE, encoder_weights='imagenet', activation='sigmoid', classes=1, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

model.load_weights('../input/unet-lung-mask-512/unet_lungseg_512_12-0.075.hdf5')
model.summary()

In [ ]:
def mask2ctr(img,im):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = 1 - img.astype(np.float32)
    img = np.where(img>=0.5, 1, 0) 
    img = img.astype(np.uint8)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    #腐蚀
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    ctr = cv2.erode(img, kernel)
    
    ctr = img - ctr
    ctr=Image.fromarray(ctr) 
    ctr = 255 * np.array(ctr).astype('uint8')
    ctr = cv2.cvtColor(ctr, cv2.COLOR_GRAY2RGB)
    
    im = im.astype(np.float32) * 255
    ctr_img = ctr + im[0,:,:,:]
    ctr_img = ctr_img.astype(np.float32) / 255
    
    return ctr,ctr_img


In [ ]:
from PIL import Image  

SIUID = test.loc[1234,'StudyInstanceUID']
im = cv2.imread(TEST_PATH+SIUID+'.jpg') 
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = im.astype(np.float32) # / 255.
im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
im1 = (im - np.min(im)) / (np.max(im) - np.min(im))
im = im1.reshape(1,IMG_WIDTH, IMG_HEIGHT, -1)

y_hat = model.predict(im)
y_hat = y_hat.squeeze(0)
_, ctr_img = mask2ctr(y_hat,im)

# fix, ax = plt.subplots(1,2, figsize=(20,10))
# ax[0,0].imshow(im)
# ax[0,1].imshow(ctr)
# plt.show()    

In [ ]:
print(im.mean())
plt.imshow(im[0,:,:,:])
plt.show()

In [ ]:
# im = im.astype(np.float32) * 255
# ctr_img = ctr + im[0,:,:,:]
# ctr_img = ctr_img.astype(np.float32) / 255

# ctr =  ctr.astype(np.float32)  / 255.
# ctr_img = ctr + im[0,:,:,:]
# ctr_img = (ctr_img - np.min(ctr_img)) / (np.max(ctr_img) - np.min(ctr_img))

In [ ]:
print(ctr_img.mean())
plt.imshow(ctr_img)
plt.show()

In [ ]:
for idx in tqdm(range(len(test))):
    SIUID = test.loc[idx,'StudyInstanceUID']
    im = cv2.imread(TEST_PATH+SIUID+'.jpg',-1)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    im = im.astype(np.float32) # / 255.
    im = cv2.resize(im, dsize=(IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    im = (im - np.min(im)) / (np.max(im) - np.min(im))
    im = im.reshape(1,IMG_WIDTH, IMG_HEIGHT, -1)
    y_hat = model.predict(im)
    y_hat = y_hat.squeeze(0)
    _, y_ctr = mask2ctr(y_hat,im)
    cv2.imwrite('./test_ctrlung/'+SIUID+'.jpg', y_ctr)

## Split folds
Ideas:
1. make sure that the labels are stratified
2. one patient's images are grouped in one fold

In [ ]:
class RepeatedStratifiedGroupKFold():

    def __init__(self, n_splits=5, n_repeats=1, random_state=None):
        self.n_splits = n_splits
        self.n_repeats = n_repeats
        self.random_state = random_state
        
    def split(self, X, y=None, groups=None):
        k = self.n_splits
        def eval_y_counts_per_fold(y_counts, fold):
            y_counts_per_fold[fold] += y_counts
            std_per_label = []
            for label in range(labels_num):
                label_std = np.std(
                    [y_counts_per_fold[i][label] / y_distr[label] for i in range(k)]
                )
                std_per_label.append(label_std)
            y_counts_per_fold[fold] -= y_counts
            return np.mean(std_per_label)
            
        rnd = check_random_state(self.random_state)
        for repeat in range(self.n_repeats):
            labels_num = np.max(y) + 1
            y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
            y_distr = Counter()
            for label, g in zip(y, groups):
                y_counts_per_group[g][label] += 1
                y_distr[label] += 1

            y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
            groups_per_fold = defaultdict(set)
        
            groups_and_y_counts = list(y_counts_per_group.items())
            rnd.shuffle(groups_and_y_counts)

            for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
                best_fold = None
                min_eval = None
                for i in range(k):
                    fold_eval = eval_y_counts_per_fold(y_counts, i)
                    if min_eval is None or fold_eval < min_eval:
                        min_eval = fold_eval
                        best_fold = i
                y_counts_per_fold[best_fold] += y_counts
                groups_per_fold[best_fold].add(g)
            
            all_groups = set(groups)
            for i in range(k):
                train_groups = all_groups - groups_per_fold[i]
                test_groups = groups_per_fold[i]

                train_indices = [i for i, g in enumerate(groups) if g in train_groups]
                test_indices = [i for i, g in enumerate(groups) if g in test_groups]

                yield train_indices, test_indices

In [ ]:
# let's first concat all the labels 
# e.g 00000000010   
target_cols = train.drop(['StudyInstanceUID', 'PatientID'],axis=1).columns.values.tolist()
targets = train[target_cols].astype(str)
# create a new col to store the label
train['combined_tar'] = ''
for i in tqdm(range(targets.shape[1])):
    train['combined_tar'] += targets.iloc[:,i]
# take a look at it
train.combined_tar.value_counts()

In [ ]:
train['combined_tar'] = LabelEncoder().fit_transform(train['combined_tar'])
len(train['combined_tar'].unique())

In [ ]:
target_cols

In [ ]:
targets

In [ ]:
X=train.values
y=targets.values

In [ ]:
y.shape

In [ ]:
train['fold'] = -1
rskf = RepeatedStratifiedGroupKFold(n_splits=nfold, random_state=seed)
for i, (train_idx, valid_idx) in enumerate(rskf.split(train, train.combined_tar, train.PatientID)): #(df, targets, group)
    train.loc[valid_idx, 'fold'] = int(i)

# !pip install /kaggle/input/iterative-stratification/iterative-stratification-master/
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
# mskf = MultilabelStratifiedKFold(n_splits=nfold, random_state=seed)
# for i, (train_idx, valid_idx) in enumerate(mskf.split(X,y)): #(df, targets, group)
#     train.loc[valid_idx, 'fold'] = int(i)

## Sanity Check
You wanna make sure this split makes sense. We can do that by checking the stratification and groups.

In [ ]:
train.query('fold==0').combined_tar.value_counts()

In [ ]:
train.query('fold==1').combined_tar.value_counts()

In [ ]:
train.query('fold==2').combined_tar.value_counts()

It seems that the label is very nicely stratified. Now let's check groups.

In [ ]:
np.intersect1d(train.query('fold==0').PatientID.unique(), train.query('fold==1').PatientID.unique())

In [ ]:
np.intersect1d(train.query('fold==1').PatientID.unique(), train.query('fold==2').PatientID.unique())

In [ ]:
np.intersect1d(train.query('fold==2').PatientID.unique(), train.query('fold==3').PatientID.unique())

No patient has appeared in two folds.

## Save final CSV

In [ ]:
train.drop('combined_tar', axis=1)


In [ ]:
train.drop('combined_tar', axis=1).to_csv('train_ctrlung_rskfolds.csv', index=False)